# Iris

## Data


In [1]:
import theano
import theano.tensor as T
import numpy as np
from sklearn.datasets import load_iris
from theano.sandbox import cuda

theano.config.floatX = 'float32'
cuda.use('gpu')
theano.config.compute_test_value = 'off'

iris = load_iris()

data_input    = np.asarray(iris.data, dtype=theano.config.floatX)
data_target   = iris.target_names[iris.target]
classes_names = iris.target_names

Using gpu device 0: GeForce GT 630M (CNMeM is enabled with initial size: 75.0% of memory, CuDNN not available)


## Training

In [2]:
import sys
import time

sys.path.insert(0, r'../../')

from libml.models.sequential import Sequential
from libml.layers.dense import Dense
from libml.utils.cost_functions import *
from libml.utils.update_functions import *
from libml.utils.regularizer_function import *
from libml.utils.metrics.classifiermetrics import ClassifierMetrics
from sklearn.cross_validation import ShuffleSplit

classifier = Sequential(classes_names, "classifier")
classifier.add_layer(Dense(n_input=data_input.shape[1], n_output=3, activation=T.tanh))
classifier.add_layer(Dense(n_output=len(classes_names), activation=T.nnet.softmax))
classifier.append_cost(mse)
classifier.append_reg(L1, lamb=0.005)
classifier.append_reg(L2, lamb=0.001)
classifier.set_update(adagrad, initial_learning_rate=0.15)
classifier.compile()

folds = 4
sss = ShuffleSplit(data_input.shape[0], n_iter=folds, test_size=None, train_size=0.6, random_state=0)
max_epoch = 400
validation_jump = 5

# Initialize metrics
metrics = ClassifierMetrics(classes_names)

for i, (train_set, test_set) in enumerate(sss):                      
    # data train and test
    input_train = data_input[train_set]
    input_test = data_input[test_set]
    target_train = data_target[train_set]
    target_test = data_target[test_set]

    # training
    tic = time.time()
    train_cost = classifier.fit(input_train, target_train,
                                max_epoch=max_epoch, batch_size=32,
                                validation_jump=validation_jump, early_stop_th=4)
    toc = time.time()

    # Compute metrics
    metrics.append_prediction(target_test, classifier.predict(input_test))
    metrics.append_cost(train_cost)
    
    # Reset parameters
    classifier.reset()

    print("%d Elapsed time [s]: %f" % (i, toc - tic))

print("FINISHED!")

ImportError: No module named 'libml.utils.regularizer_function'

## Results

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

metrics.print()
metrics.plot_confusion_matrix()
metrics.plot_cost(max_epoch)

plt.tight_layout()